In [5]:
import ast
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

california_hotels = pd.read_csv("files/data/booking/california_hotels.csv", index_col=0)
california_hotels.dropna(subset=["attributes"], inplace=True)
california_hotels['attributes'] = california_hotels['attributes'].apply(ast.literal_eval)
california_hotels


,state,city,name,avg_score,price,stars,reviews_url,attributes,latitude,longitude,Personal,Instalaciones y servicios,Limpieza,Confort,Relación calidad-precio,Ubicación,WiFi Gratis
1,California,Los Angeles,W Hollywood,73.0,849.752,4.0,https://www.booking.com/hotel/us/w-hollywood.e...,"[Habitaciones sin humo, 4 restaurantes, WiFi e...",34.100794,-118.325906,9.4,8.5,9.0,9.0,8.5,9.2,9.1
2,California,Los Angeles,Park Plaza Lodge,86.0,466.927,3.0,https://www.booking.com/hotel/us/park-plaza-lo...,"[Restaurante, Parking gratis, Tetera/cafetera ...",34.070129,-118.351360,8.7,8.2,8.5,8.6,7.2,8.6,8.5
3,California,Los Angeles,Moxy Downtown Los Angeles,79.0,904.364,4.0,https://www.booking.com/hotel/us/moxy-downtown...,"[WiFi gratis, Desayuno, Terraza, Parking priva...",34.041151,-118.267341,9.0,9.0,9.1,9.0,8.8,9.0,9.2
4,California,Los Angeles,Hollywood Homes minutes to everything SPACIOUS...,88.0,463.104,4.0,https://www.booking.com/hotel/us/hollywood-cos...,"[Habitaciones familiares, Aire acondicionado, ...",34.086662,-118.314868,8.1,7.2,7.7,7.7,7.0,8.4,8.3
5,California,Los Angeles,Best Western Hollywood Plaza Inn Hotel - Holly...,72.0,390.267,3.0,https://www.booking.com/hotel/us/best-western-...,"[Habitaciones sin humo, Restaurante, Adaptado ...",34.107181,-118.337637,8.3,8.4,8.7,8.8,7.6,8.5,8.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,California,Bakersfield,"Country Inn & Suites by Radisson, Bakersfield, CA",76.0,233.351,2.0,https://www.booking.com/hotel/us/country-inn-s...,"[Habitaciones sin humo, Adaptado personas de m...",35.334177,-119.038601,10.0,9.5,9.5,9.5,9.5,10.0,NaN
1222,California,Bakersfield,La Quinta by Wyndham Bakersfield North,79.0,264.282,3.0,https://www.booking.com/hotel/us/la-quinta-spe...,"[Piscina interior, WiFi gratis, ¿Se sirven des...",35.440505,-119.075682,9.1,7.6,8.7,7.6,8.2,9.1,8.0
1223,California,Bakersfield,"Hourglass Hotel, Ascend Hotel Collection",82.0,230.620,3.0,https://www.booking.com/hotel/us/the-hourglass...,"[WiFi gratis, Desayuno, ¿Se sirven desayunos?,...",35.380412,-119.046076,8.8,8.0,8.8,8.2,7.7,9.0,NaN
1224,California,Bakersfield,SpringHill Suites Bakersfield,84.0,270.872,3.0,https://www.booking.com/hotel/us/springhill-su...,"[Piscina interior, WiFi gratis, Desayuno, ¿Se ...",35.380008,-119.049611,8.4,6.9,8.4,7.8,7.6,7.4,6.3


In [7]:
# crear dataframe con atributos unicos
california_hotels_attributes_exploded = california_hotels.explode("attributes")
california_hotels_attributes_exploded

atributtes_list = california_hotels_attributes_exploded["attributes"].tolist()
unique_attributes = list(set(atributtes_list))

filtered_attributes = []
for attr in unique_attributes:
    if not pd.Series(attr).str.contains("\n").any() and "?" not in attr:
        filtered_attributes.append(attr)

attributes_df = pd.DataFrame(columns=list(filtered_attributes))

# Create dummies
for index, attributes_list in enumerate(california_hotels['attributes']):
    for attr in filtered_attributes:
        attributes_df.loc[index, attr] = 1 if attr in attributes_list else 0

attributes_df

,Parque acuático,Inglés,Hervidor eléctrico,12 restaurantes,Parking privado,WC,Terraza,Spa y centro de bienestar,Registro de entrada / salida privado,Zona de pícnic,...,Tetera/cafetera en todas las habitaciones,2 piscinas,Canales por cable,Habitaciones sin humo,Ducha,Secadora de ropa,Restaurante,Tostadora,Español,WiFi en todo el alojamiento
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0
2,0,0,0,1,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1165,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
1166,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1167,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [8]:
attributes_matrix = attributes_df.T
cosine_similarity_matrix = cosine_similarity(attributes_df)
cosine_similarity_matrix

array([[1.        , 0.4       , 0.5       , ..., 0.63245553, 0.55901699,
        0.6       ],
       [0.4       , 1.        , 0.4       , ..., 0.52704628, 0.3354102 ,
        0.7       ],
       [0.5       , 0.4       , 1.        , ..., 0.63245553, 0.67082039,
        0.6       ],
       ...,
       [0.63245553, 0.52704628, 0.63245553, ..., 1.        , 0.82495791,
        0.73786479],
       [0.55901699, 0.3354102 , 0.67082039, ..., 0.82495791, 1.        ,
        0.55901699],
       [0.6       , 0.7       , 0.6       , ..., 0.73786479, 0.55901699,
        1.        ]])

In [9]:
cosine_similarity_df = pd.DataFrame(cosine_similarity_matrix, index=california_hotels.index, columns=california_hotels.index)
cosine_similarity_df.to_csv("files/data/booking/california_hotels_similarity_matrix.csv")

In [10]:
cosine_similarity_df

,1,2,3,4,5,6,7,8,9,10,...,1215,1216,1217,1218,1219,1221,1222,1223,1224,1225
1,1.000000,0.400000,0.500000,0.258199,0.400000,0.600000,0.400000,0.300000,0.223607,0.129099,...,0.316228,0.119523,0.258199,0.119523,0.282843,0.600000,0.421637,0.632456,0.559017,0.600000
2,0.400000,1.000000,0.400000,0.516398,0.600000,0.600000,0.400000,0.500000,0.335410,0.387298,...,0.474342,0.239046,0.258199,0.239046,0.282843,0.600000,0.421637,0.527046,0.335410,0.700000
3,0.500000,0.400000,1.000000,0.387298,0.500000,0.600000,0.400000,0.500000,0.335410,0.387298,...,0.316228,0.239046,0.387298,0.239046,0.424264,0.600000,0.527046,0.632456,0.670820,0.600000
4,0.258199,0.516398,0.387298,1.000000,0.516398,0.387298,0.516398,0.645497,0.433013,0.666667,...,0.816497,0.308607,0.333333,0.308607,0.365148,0.645497,0.680414,0.680414,0.577350,0.387298
5,0.400000,0.600000,0.500000,0.516398,1.000000,0.500000,0.400000,0.500000,0.447214,0.258199,...,0.474342,0.239046,0.387298,0.239046,0.424264,0.700000,0.527046,0.632456,0.559017,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,0.600000,0.600000,0.600000,0.645497,0.700000,0.600000,0.400000,0.400000,0.335410,0.387298,...,0.632456,0.239046,0.387298,0.239046,0.424264,1.000000,0.737865,0.948683,0.782624,0.800000
1222,0.421637,0.421637,0.527046,0.680414,0.527046,0.527046,0.316228,0.421637,0.353553,0.408248,...,0.666667,0.251976,0.408248,0.251976,0.447214,0.737865,1.000000,0.777778,0.824958,0.527046
1223,0.632456,0.527046,0.632456,0.680414,0.632456,0.632456,0.421637,0.421637,0.353553,0.408248,...,0.666667,0.251976,0.408248,0.251976,0.447214,0.948683,0.777778,1.000000,0.824958,0.737865
1224,0.559017,0.335410,0.670820,0.577350,0.559017,0.559017,0.335410,0.447214,0.375000,0.288675,...,0.530330,0.267261,0.433013,0.267261,0.474342,0.782624,0.824958,0.824958,1.000000,0.559017


In [36]:
cosine_similarity_df.iloc[95].sort_values(ascending=False).head(25)

98      1.0
155     0.9
66      0.9
111     0.9
609     0.8
542     0.8
148     0.8
500     0.8
1120    0.8
519     0.8
524     0.8
526     0.8
40      0.8
1088    0.8
565     0.8
556     0.8
150     0.8
115     0.8
574     0.8
108     0.8
602     0.8
74      0.8
84      0.8
83      0.8
28      0.8
Name: 98, dtype: float64